# Домашнее задание "Проблема качества данных"


### Задание
Необходимо запустить практическую часть занятия, и посмотреть самому то, о чём говорили на лекции. По образу практики, попробуйте создать искусственный датасет с лишними столбцами. Целевую метку, при правильной обработке данных, формируйте таким образом, чтобы без затруднений её смогла описать линейная модель. Ориентируйтесь на то, что было показано во время занятия, и каждый шаг описывайте в markdown. Здесь важно видеть ваш ход мысли. Не бойтесь ошибиться или написать не то. Данное задание не имеет какого-то “правильного” решения. Цель - достичь базового понимания проблемы. Чем больше вы фантазируете, тем лучше :) Тем не менее, старайтесь представить те ситуации, которые по-вашему мнению могли бы быть в реальных данных.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


In [2]:
def print_ml_model_report(df, cols, target_col):

    X = df[cols]
    y = df[target_col]

    reg = LinearRegression().fit(X, y)

    print('Weights: {}'.format(reg.coef_))
    print('Bias: {}'.format(reg.intercept_))

    pred_values = reg.predict(df[cols])
    print('Error: {}'.format(mean_absolute_error(pred_values, y)))


In [3]:
# Creating cars dataset
n_samples = 10000
car_color_types = ['white', 'green', 'blue', 'red', 'black']

car_color = np.random.choice(car_color_types, n_samples)
car_length = np.random.randint(low=1200, high=5000, size=n_samples) + 26  # car_length in mm plus noise digit
car_width = np.random.uniform(low=1., high=1.5, size=n_samples) + 0.2  # car_width in meters plus noise digit
car_horsepower = np.random.randint(low=75, high=500, size=n_samples) + 12

# to calculate target convert car_length to meter
# car_horsepower convert to kiloWatts (multiply by 0.735)
# add some noise (* 25 + 226)
car_price = (car_length / 1000) * car_width * (car_horsepower * 0.735) * 25 + 226  # target

cars_df = pd.DataFrame(
    {'car_color': car_color, 'car_length (mm)': car_length, 'car_width (m)': car_width, 'car_power (horsepower)': car_horsepower, 'car_price ($)': car_price})
cars_df


,car_color,car_length (mm),car_width (m),car_power (horsepower),car_price ($)
0,blue,2512,1.366303,221,14163.545752
1,white,3851,1.608684,382,43710.538300
2,black,4191,1.217700,377,35579.045848
3,white,2636,1.233170,311,18802.173848
4,blue,3071,1.533365,332,28953.027555
...,...,...,...,...,...
9995,red,2668,1.699830,90,7725.996317
9996,blue,4862,1.487922,473,63101.812064
9997,green,3956,1.492615,483,52631.701091
9998,green,1713,1.453423,181,8506.476730


In [4]:
# print report, bad results...
cols = ['car_length (mm)', 'car_width (m)', 'car_power (horsepower)']
print_ml_model_report(cars_df, cols, 'car_price ($)')


Weights: [7.98996714e+00 1.70819804e+04 8.23979598e+01]
Bias: -49193.28074881459
Error: 2846.241993996496


In [5]:
# let's improve model, try to use categorial column 'car_color'
# convert categorial data to numbers using by one hot encoding
encoder = OneHotEncoder()
encoder_results = encoder.fit_transform(cars_df[['car_color']]).toarray()
enc_df = pd.DataFrame(encoder_results, columns=encoder.categories_)
encoded_df = cars_df.join(enc_df)

encoded_df = encoded_df.drop('car_color', 1)
# encoded_df.head()

# print report - as we can see from the report, it got worse
print_ml_model_report(encoded_df, encoded_df.columns, 'car_price ($)')


Weights: [ 1.01549174e-15 -2.19928661e-11 -2.31483859e-14  1.00000000e+00
  5.62779749e-13  4.62832467e-15 -3.15619577e-13 -9.33164496e-13
  6.81376003e-13]
Bias: 1.8189894035458565e-11
Error: 7.129847290343605e-12


In [6]:
# let's try to add new columns
# convert 'car_length' to meter
cars_df['car_length (m)'] = cars_df['car_length (mm)'] / 1000

# print report - not the best
print_ml_model_report(cars_df, ['car_length (m)', 'car_width (m)', 'car_power (horsepower)'], 'car_price ($)')


Weights: [ 7989.96714236 17081.98044143    82.39795977]
Bias: -49193.28074882693
Error: 2846.2419939966067


In [7]:
# convert 'car_power' to kiloWatts
cars_df['car_power (kiloWatts)'] = cars_df['car_power (horsepower)'] * 0.735

# print report - the same results
print_ml_model_report(cars_df, ['car_length (m)', 'car_width (m)', 'car_power (kiloWatts)'], 'car_price ($)')


Weights: [ 7989.96714236 17081.98044143   112.10606771]
Bias: -49193.2807488238
Error: 2846.241993996579


In [8]:
# let's try to use StandartScaler()
scaler = StandardScaler()
to_scale_df = cars_df[[
    'car_length (m)', 'car_width (m)', 'car_power (kiloWatts)', 'car_price ($)']]
scaler.fit(to_scale_df)

scaler_df = pd.DataFrame(scaler.transform(
    to_scale_df), columns=to_scale_df.columns)

# print report - not the best again
print_ml_model_report(scaler_df, [
                      'car_length (m)', 'car_width (m)', 'car_power (kiloWatts)'], 'car_price ($)')


Weights: [0.616179   0.17450434 0.70816096]
Bias: 7.748056944300584e-16
Error: 0.2002432278750497


In [9]:
# let's try to add 'multi' column - multiply 'car_length (m)', 'car_width (m)', 'car_power (kiloWatts)' cols
cars_df['multi'] = cars_df['car_length (m)'] * cars_df['car_width (m)'] * cars_df['car_power (kiloWatts)']

# print report - looks now much better
print_ml_model_report(cars_df, ['multi'], 'car_price ($)')


Weights: [25.]
Bias: 226.00000000001455
Error: 6.7067958298139275e-12
